In [1]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn
import cv2

import glob

In [2]:
# unzip Dataset

from zipfile import ZipFile
file_name='/content/drive/MyDrive/freelance_projects/insects/insects.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [3]:
dataset = "TRAIN"
output = "/content/drive/MyDrive/freelance_projects/insects/out"

In [5]:
# import splitfolders  # or import split_folders

# # Split with a ratio. (80% training, 10% val, 10% test)
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(dataset, output=output, seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

# # Split val/test with a fixed number of items e.g. 100 for each set.
# # To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
# splitfolders.fixed(dataset, output=output, seed=1337, fixed=(100, 100), oversample=False, group_prefix=None) # default values

In [6]:
train_dir = '/content/drive/MyDrive/freelance_projects/insects/out/train'
test_dir = '/content/drive/MyDrive/freelance_projects/insects/out/test'
val_dir = '/content/drive/MyDrive/freelance_projects/insects/out/val'

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale = 1./255.,)
test_datagen = ImageDataGenerator(rescale = 1./255.,)


train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='categorical', target_size = (220, 220))
validation_generator = val_datagen.flow_from_directory(val_dir, batch_size=20, class_mode = 'categorical', target_size=(220, 220))
test_generator = test_datagen.flow_from_directory(test_dir, batch_size=20, class_mode = 'categorical', target_size=(220, 220))

Found 1078 images belonging to 9 classes.
Found 134 images belonging to 9 classes.
Found 143 images belonging to 9 classes.


In [14]:
train_generator.class_indices

{'ARMYWORMS': 0,
 'ASPHIDS': 1,
 'BEETLE': 2,
 'BOLLWORMS': 3,
 'GRASSHOPPERS': 4,
 'MITES': 5,
 'MOSQUITO': 6,
 'SAW FLY': 7,
 'STEM BORER': 8}

In [15]:
input_shape = (220, 220, 3)

In [16]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape=input_shape, include_top=False)

for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 220, 220, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 226, 226, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 110, 110, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

In [26]:
from keras import models, regularizers
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(9, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_20 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               65664     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                      

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch = len(train_generator),
                    epochs=2,
                    validation_steps=len(validation_generator))

Epoch 1/2
54/54 [==============================] - 119s 2s/step - loss: 2.3898 - accuracy: 0.4601 - val_loss: 1.8735 - val_accuracy: 0.6269
Epoch 2/2
54/54 [==============================] - 116s 2s/step - loss: 2.0220 - accuracy: 0.5733 - val_loss: 1.7601 - val_accuracy: 0.6119
